In [1]:
import pandas as pd
import numpy as np
import torch
import transformers
import pickle
import os

import nlpsig
import nlpsig_networks

from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from nlpsig.classification_utils import DataSplits, Folds
from nlpsig_networks.pytorch_utils import SaveBestModel, training_pytorch, testing_pytorch, set_seed, KFold_pytorch
from nlpsig_networks.focal_loss import FocalLoss
from nlpsig_networks.ffn import FeedforwardNeuralNetModel
from nlpsig_networks.deepsignet import StackedDeepSigNet

from tqdm.notebook import tqdm

seed = 2023

In [2]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    implement_ffn,
    ffn_hyperparameter_search,
    obtain_mean_history,
    obtain_signatures_history,
    histories_baseline_hyperparameter_search
)
from nlpsig_networks.scripts.sdsn_functions import (
    obtain_SDSN_input,
    sdsn_hyperparameter_search
)

In [3]:
import signatory

In [4]:
signatory.signature_channels(channels=16, depth=4)

69904

In [5]:
signatory.logsignature_channels(in_channels=16, depth=4)

17816

In [6]:
output_dir = "client_talk_type"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [7]:
anno_mi = pd.read_csv("AnnoMI-full.csv")
anno_mi["datetime"] = pd.to_datetime(anno_mi["timestamp"])
anno_mi = anno_mi.drop(columns=["video_title", "video_url"])
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-06-07 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-07 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-07 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-07 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-07 00:00:34


In [8]:
anno_mi["interlocutor"].value_counts()

therapist    6826
client       6725
Name: interlocutor, dtype: int64

In [9]:
anno_mi["main_therapist_behaviour"].value_counts() / anno_mi["interlocutor"].value_counts()["therapist"]

other              0.313947
question           0.286258
reflection         0.251538
therapist_input    0.148257
Name: main_therapist_behaviour, dtype: float64

In [10]:
anno_mi["client_talk_type"].value_counts() / anno_mi["interlocutor"].value_counts()["client"]

neutral    0.627063
change     0.248030
sustain    0.124907
Name: client_talk_type, dtype: float64

In [11]:
anno_mi["interlocutor"].value_counts()

therapist    6826
client       6725
Name: interlocutor, dtype: int64

In [12]:
anno_mi["topic"].value_counts()

reducing alcohol consumption                                                          2326
more exercise / increasing activity                                                   2034
reducing recidivism                                                                   1303
reducing drug use                                                                     1104
diabetes management                                                                    948
smoking cessation                                                                      923
smoking cessation                                                                      541
taking medicine / following medical procedure                                          448
asthma management                                                                      431
avoiding DOI                                                                           394
changing approach to disease                                                           315

In [13]:
len(anno_mi["transcript_id"].unique())

133

## Only considering client for now...

In [14]:
client_index = [isinstance(x, str) for x in anno_mi["client_talk_type"]]
sum(client_index)

6725

In [15]:
y_data = anno_mi["client_talk_type"][client_index]
y_data.shape

(6725,)

In [16]:
y_data[0:20]

1     neutral
3     neutral
5     neutral
7     neutral
9     neutral
11    neutral
13    neutral
15    neutral
17    neutral
19    neutral
21    neutral
23    neutral
25    neutral
27    neutral
29    neutral
31    neutral
33    neutral
35     change
37     change
39     change
Name: client_talk_type, dtype: object

In [17]:
label_to_id = {y_data.unique()[i]: i for i in range(len(y_data.unique()))}
id_to_label = {v: k for k, v in label_to_id.items()}

In [18]:
label_to_id

{'neutral': 0, 'change': 1, 'sustain': 2}

In [19]:
id_to_label

{0: 'neutral', 1: 'change', 2: 'sustain'}

In [20]:
y_data = [label_to_id[x] for x in y_data]
y_data[0:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]

In [21]:
output_dim = len(label_to_id.values())
output_dim

3

## Obtaining SBERT Embeddings

We can use the `SentenceEncoder` class within `nlpsig` to obtain sentence embeddings from a model. This class uses the [`sentence-transformer`](https://www.sbert.net/docs/package_reference/SentenceTransformer.html) package and here, we have use the pre-trained `all-MiniLM-L12-v2` model by passing this name as a string to the class - alternative models can be found [here](https://www.sbert.net/docs/pretrained_models.html).

We can pass our dataframe and the column name which stores the sentences that we wish to encode along with the model name into the constructor of the class to initialise our sentence encoder as follows:

In [22]:
# initialise the Text Encoder
sentence_encoder = nlpsig.SentenceEncoder(df=anno_mi,
                                          feature_name="utterance_text",
                                          model_name="all-MiniLM-L12-v2")

We used the `.load_pretrained_model()` method to load in the pre-trained model - this may require you to download the model if this is the first time running the notebook.

In [23]:
sentence_encoder.load_pretrained_model()

We can then obtain embeddings via the `.obtain_embeddings()` method.

In [24]:
sbert_embeddings = sentence_encoder.obtain_embeddings()

[INFO] number of sentences to encode: 13551


Batches:   0%|          | 0/212 [00:00<?, ?it/s]

In [25]:
sbert_embeddings.shape

(13551, 384)

We can save our embeddings for use later:

In [26]:
np.save(f"{output_dir}/anno_mi_client_sentence_embeddings_384",
        sbert_embeddings)

In [27]:
sbert_embeddings = np.load(f"{output_dir}/anno_mi_client_sentence_embeddings_384.npy")

# Baseline: FFN baseline

Using the embeddings for the sentences directly in a FFN to predict the client talk type.

Going to try out some variations (1 hidden layer, 2 hidden layers and 3 hidden layers - all of size 100)

In [28]:
num_epochs = 100
hidden_dim_sizes = [[100]*i for i in range(1, 4)]
dropout_rates = [0.5, 0.2, 0.1]
learning_rates = [5e-3, 1e-3, 5e-4, 1e-4, 1e-5]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [29]:
hidden_dim_sizes

[[100], [100, 100], [100, 100, 100]]

In [30]:
learning_rates

[0.005, 0.001, 0.0005, 0.0001, 1e-05]

In [ ]:
ffn_current, best_ffn_current, best_valid_metric, extra_info = ffn_hyperparameter_search(
    num_epochs=num_epochs,
    x_data=sbert_embeddings[client_index],
    y_data=y_data,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=f"ffn_current_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
ffn_current

In [ ]:
best_ffn_current

In [ ]:
ffn_current_kfold, best_ffn_current_kfold, best_valid_metric, extra_info = ffn_hyperparameter_search(
    num_epochs=num_epochs,
    x_data=sbert_embeddings[client_index],
    y_data=y_data,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"ffn_current_focal_{gamma}_kfold.csv",
    verbose=False
)

In [ ]:
ffn_current_kfold

In [ ]:
best_ffn_current_kfold

# Baseline 2: Averaging history and use FFN

Here, we will use `nlpsig` to construct some paths of embeddings for the last $k$ utterances which we will average and use those in a FFN. We will also concatenate the current utterance embedding to the FFN input - all of this is done in the `obtain_mean_history` function in `nlpsig-networks.scripts.ffn_baseline_functions` which we imported earlier on.

Here, we will run the hyperparameter search to implement the FFN with the same parameters as above. For this baseline, we can also see how well the model performance is affected by the size of the history window $k$.

In [ ]:
window_sizes = [5, 10, 20, 50]

In [ ]:
ffn_mean_history, best_valid_metric, extra_info = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_mean_history_focal_{gamma}.csv",
    verbose=False
)

In [ ]:
ffn_mean_history_kfold, best_valid_metric, extra_info = histories_baseline_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data,
    output_dim=output_dim,
    window_sizes=window_sizes,
    hidden_dim_sizes=hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    use_signatures=False,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    path_indices=client_index,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_mean_history_focal_{gamma}_kfold.csv",
    verbose=False
)

# Baseline 3: FFN using signatures

First, we dimension reduce these and then take signatures. We use the path signature as input to the FFN for classification.

In [286]:
dimensions_and_sig_depths = [(30, 2), (10, 3), (5, 4), (4, 5), (3, 6)]
dim_reduce_methods = ["gaussian_random_projection", "umap"]

In [171]:
[signatory.signature_channels(channels, depth)
 for (channels, depth) in dimensions_and_sig_depths]

[30, 900]
[10, 100, 1000]
[5, 25, 125, 625]
[4, 16, 64, 256, 1024]
[3, 9, 27, 81, 243, 729]


[930, 1110, 780, 1364, 1092]

In [ ]:
[signatory.logsignature_channels(channels, depth)
 for (channels, depth) in dimensions_and_sig_depths]

# Baseline: LSTM classification

# Baseline: Fine-tune BERT for classification

# StackedDeepSigNet

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [244]:
time_features = ["time_encoding", "timeline_index"]
path_specifics = {"pad_by": "history",
                  "zero_padding": True,
                  "method": "k_last",
                  "k": 10,
                  "time_feature": time_features,
                  "standardise_method": ["minmax", None],
                  "embeddings": "dim_reduced",
                  "include_current_embedding": True,
                  "pad_from_below": False}

In [298]:
sig_depths = [2,3]
dim_reduce_methods = ["gaussian_random_projection", "umap"]

In [297]:
output_dim = len(label_to_id)
lstm_hidden_dims = [[8,8], [12,12,8]]
num_time_features = len(time_features)
conv_output_channels = [20, 10, 5]
learning_rate = 1e-4

In [300]:
embedding_dim = 384
dimensions = [embedding_dim, 100, 50, 30]